In [1]:
# Import libs
import os
import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# Load dataset
# get data_dir for tranining
current_path = os.getcwd()

img_height = 224
img_width = 224
batch_size = 64

# set train dataset
train_data_dir = os.path.join(current_path, 'Vegetable_Images\\train')
train_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# set validation dataset
val_data_dir = os.path.join(current_path, 'Vegetable_Images\\validation')
val_ds = keras.utils.image_dataset_from_directory(
    val_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

class_names = train_ds.class_names
class_num = len(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [3]:
# Normalize data
data_scaling = keras.layers.Rescaling(scale=1./255)

train_ds = train_ds.map( lambda x, y: (data_scaling(x), y))
val_ds = val_ds.map( lambda x, y: (data_scaling(x), y))

In [4]:
# Buil model
InceptionV3_excludeTop = keras.applications.InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
InceptionV3_excludeTop.trainable = False

Top_layers = keras.layers.GlobalAveragePooling2D()(InceptionV3_excludeTop.output)
Top_layers = keras.layers.Dense(units=1024, activation='relu', use_bias=True)(Top_layers)
Top_layers = keras.layers.Dropout(rate=0.25)(Top_layers)
Top_layers = keras.layers.Dense(units=class_num, activation='softmax', use_bias=True)(Top_layers)

InceptionV3_Model = keras.Model(inputs=InceptionV3_excludeTop.input, outputs=Top_layers)

InceptionV3_Model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

InceptionV3_Model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 23,916,335 (91.23 MB)

 Trainable params: 2,113,551 (8.06 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [5]:
# Train model
logdir = os.path.join("logs", datetime.datetime.now().strftime("InceptionV3_Model-%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = InceptionV3_Model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 339s 1s/step - accuracy: 0.7895 - loss: 0.7753 - val_accuracy: 0.9850 - val_loss: 0.0693
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 346s 1s/step - accuracy: 0.9850 - loss: 0.0715 - val_accuracy: 0.9923 - val_loss: 0.0384
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 345s 1s/step - accuracy: 0.9924 - loss: 0.0391 - val_accuracy: 0.9913 - val_loss: 0.0302
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 346s 1s/step - accuracy: 0.9942 - loss: 0.0249 - val_accuracy: 0.9943 - val_loss: 0.0215
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 347s 1s/step - accuracy: 0.9972 - loss: 0.0176 - val_accuracy: 0.9927 - val_loss: 0.0217
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.9974 - loss: 0.0141 - val_accuracy: 0.9970 - val_loss: 0.0141
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 351s 1s/step - accuracy: 0.9990 - loss: 0.0094 - val_accuracy: 0.9960 - val_loss: 0.0138
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 351s 1s/step - accuracy: 0.9987 - loss: 0.0077 - val_accu

In [6]:
# Save model as keras
InceptionV3_Model.save('saved_models\InceptionV3_Model.keras')